# SEGY handling in julia

In this tutorial we will show we handle SEGY data in julia:
- How t o read a SEGY file
- How to scan multiple SEGY files
- How JUDI interfaces with SegyIO

In [1]:
using JUDI, SegyIO, PyPlot

# Data

We consider a subset of the 2007 BP TTI dataset. 

In [ ]:
run(`curl -L -o data.zip https://www.dropbox.com/sh/m2ksfp7wjww3jds/AADswzf_8ZdMiDW-nmrLNgJ0a?dl=0`)

In [22]:
readdir("/data/bpttidata/")

LoadError: IOError: readdir("/data/bpttidata/"): no such file or directory (ENOENT)

## SegyRead

We first red a single file directly into memory.

This data is a segy container. We can access different headers and the traces:
- `shot.data` contains the traces (in IBMFloat32 format)
- `shot.fileheader` contains the file header
- `shot.traceheader[i]` containts thew trace heaader for the trace numer `i`

In [3]:
shot = segy_read("../data/bpttidata/BPTTI_1289.segy");

LoadError: SystemError: opening file "../data/bpttidata/BPTTI_1289.segy": No such file or directory

In [4]:
shot.fileheader

LoadError: UndefVarError: shot not defined

In [5]:
shot.traceheaders[10]

LoadError: UndefVarError: shot not defined

We can then easily extra headers from the segy container

In [6]:
sx = get_header(shot, "SourceX")
rx = get_header(shot, :GroupX);
dt = get_header(shot, :dt)[1]*1e-3
T = (get_header(shot, :ns)[1] -1)*dt

LoadError: UndefVarError: shot not defined

In [7]:
extent = [rx[1], rx[end], T, 0]
figure(figsize=(10, 10))
imshow(Float32.(shot.data), vmin=-.5, vmax=.5, cmap="PuOr", interpolation=:hamming, extent=extent)
xlabel("Receiver positon (m)")
ylabel("Time (ms)")
title("Source at $(sx[1])m")

LoadError: UndefVarError: rx not defined

## Segy Scan

The scanner is designed to be multi-scale, meaning that your workflow will stay the same regardless of whether you are working on 1 MB or 1TB of data. SegyIO accomplishes this by being:

  - Direct. Read only what needs to be read.

  - Seamless. Remove the need to deal with complex filesystems.

  - Performant. Read, write, and scan at disk speed.

A scanned volume provides a higher level of abstraction, removing the need for a user to directly manage individual files. Scanning a file (or a group of files) returns a SeisCon object, which contains the necessary information to partition the volume into more managable pieces and directly access these partitions. By default, the scanner will automatically partition the volume when the source location changes.


In [8]:
dir2scan = joinpath(pwd(),"../data/bpttidata/")
file_filter = "BPTTI"

"BPTTI"

We choose wich keys we want to get easy access to after the scan.

In [9]:
keys = ["GroupX", "GroupY", "RecGroupElevation", "SourceSurfaceElevation", "dt"]

5-element Vector{String}:
 "GroupX"
 "GroupY"
 "RecGroupElevation"
 "SourceSurfaceElevation"
 "dt"

In [10]:
segy_lu = segy_scan(dir2scan, file_filter, keys);

LoadError: IOError: readdir("/data/home/zyin62/Desktop/projects/SLIMTutorials/../data/bpttidata/"): no such file or directory (ENOENT)

As we can see we found all the segy files in the directory. We know there is 111 shot records let's check the container

In [11]:
length(segy_lu)

LoadError: UndefVarError: segy_lu not defined

We scanned correctly the directory for all the shot records. We can now extract global information such as all the source positions

In [12]:
src_locations = get_sources(segy_lu).*1e-4

LoadError: UndefVarError: segy_lu not defined

# judiVector
Finally, we can wrap the segy container into a judiVector to use it for example for inversion

In [13]:
d_tti = judiVector(segy_lu);

LoadError: UndefVarError: segy_lu not defined

In [14]:
d_tti.nsrc

LoadError: UndefVarError: d_tti not defined

And we can create a source object with unknow wavelet (initialize to a  wavelet in this case)

In [15]:
src_geometry = Geometry(segy_lu; key="source")
wavelet = ricker_wavelet(src_geometry.t[1], src_geometry.dt[1], 0.015)  # 15 Hz peak frequency
q = judiVector(src_geometry, wavelet)

LoadError: UndefVarError: segy_lu not defined

In [16]:
shot_id = 47
shot = get_data(d_tti[47])
xrec = shot.geometry.xloc[1]
xsrc = Geometry(q[47].geometry).xloc[1] # convert the out of core geometry to an incore geometry to read the value

LoadError: UndefVarError: d_tti not defined

In [17]:
extent = [xrec[1], xrec[end], src_geometry.t[1], 0]
figure(figsize=(10, 10))
imshow(shot.data[1], vmin=-.5, vmax=.5, cmap="PuOr", interpolation=:hamming, extent=extent)
xlabel("Receiver positon (m)")
ylabel("Time (ms)")
title("Source at $(sx[1])m")

LoadError: UndefVarError: xrec not defined